In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import cactas as C
import tensorflow as tf
import pickle
import mahotas as mh
from sklearn.model_selection import KFold
from keras import backend as K
import nrrd
from sklearn.model_selection import KFold, train_test_split

2024-10-01 01:02:14.869828: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
DATAPATH='/raid/mpsych/CACTAS/DATA/ESUS'
CAPATH='/raid/mpsych/CACTAS/DATA/CA_ESUS3'

In [4]:
# Load files

In [5]:
images, labels = C.Helper.load_data(DATAPATH)

In [6]:
masks = C.Helper.load_seg_data(CAPATH)

In [7]:
#len(images), len(labels), len(masks)

In [8]:
# list data in same order

In [9]:
numbers = [filename.split(".")[0] for filename in images]
label_dict = {filename.split(".")[0]: filename for filename in labels}
mask_dict = {filename.split(".")[0]: filename for filename in masks}

data = [(filename, label_dict[filename.split(".")[0]], mask_dict[filename.split(".")[0]]) for filename in images]

In [10]:
#len(numbers)

In [11]:
X_train = []
y_train = []
m_train = []

In [12]:
data.sort(key=lambda x: (x[0].split(".")[0] in numbers, random.random()))

for image, label, mask in data:
    if image.split(".")[0] in numbers:
        X_train.append(image)
        y_train.append(label)
        m_train.append(mask)

In [13]:
num_patients = len(images)

In [14]:
num_patients

70

In [15]:
img = []
label = []
mask = []

In [16]:
# read and normalize data
for file in X_train:
    data, header = nrrd.read(DATAPATH + "/" +file)
    normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))
    img.append(normalized_data)

for file in y_train:
    data, header = nrrd.read(DATAPATH + "/" + file)
    label.append(data)

for file in m_train:
    data, header = nrrd.read(CAPATH + "/" + file)
    normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))
    mask.append(normalized_data)

In [17]:
img[0].shape, label[0].shape, mask[0].shape

((512, 512, 200), (512, 512, 200), (512, 512, 200))

In [18]:
# extract data

In [19]:
# Image
slices = []
for i in range(len(img)):
    for z in range(img[i].shape[2]):
        slice_2d = img[i][:, :, z]
        slices.append(slice_2d)
X_train_array = np.array(slices)

# Label
slices1 = []
for i in range(len(label)):
    for z in range(label[i].shape[2]):
        slice_2d = label[i][:, :, z]
        slices1.append(slice_2d)

new_slices = []
for i in range(len(slices1)):
    slices = np.where(slices1[i] != 0, True, False)
    new_slices.append(slices)
y_train_array = np.array(new_slices)

# Mask
slices_mtrain=[]
for i in range(len(mask)):
    for z in range(mask[i].shape[2]):
        slice_2d = mask[i][:, :, z]
        dilated = mh.dilate(slice_2d.astype(np.bool_))
        for _ in range(9):
            dilated = mh.dilate(dilated)
        slices_mtrain.append(dilated)
m_train_array = np.array(slices_mtrain)

In [20]:
X_train_array = X_train_array.reshape(X_train_array.shape[0], X_train_array.shape[1],X_train_array.shape[2], 1)
y_train_array = y_train_array.reshape(y_train_array.shape[0], y_train_array.shape[1],y_train_array.shape[2], 1)
m_train_array = m_train_array.reshape(m_train_array.shape[0], m_train_array.shape[1],m_train_array.shape[2], 1)

In [21]:
# print(X_train_array.shape, y_train_array.shape, m_train_array.shape)

In [22]:
# X_train_array.dtype, y_train_array.dtype, m_train_array.dtype

In [23]:
X_train_array = X_train_array.astype(np.float32)
m_train_array = m_train_array.astype(np.float32)

In [24]:
# X_train_array.dtype, y_train_array.dtype, m_train_array.dtype

In [25]:
# Mask Image
train_masks=[]
for i in range(len(m_train_array)):
    binary = (m_train_array[i] > 0).astype(np.uint8)
    train_masks.append(binary)

In [26]:
train_images=[]
for i in range(len(X_train_array)):
    train_image = X_train_array[i] * train_masks[i]
    train_images.append(train_image)
images_array = np.array(train_images) 

In [27]:
#images_array = images_array.astype(np.float32)
images_array = images_array.reshape(images_array.shape[0],images_array.shape[1],images_array.shape[2], 1)

In [28]:
slices_per_patient = [im.shape[2] for im in img]

In [29]:
# len(slices_per_patient)

In [30]:
#slices_per_patient

In [31]:
kf = KFold(n_splits=num_patients // 5)

In [32]:
kf

KFold(n_splits=14, random_state=None, shuffle=False)

In [33]:
for train_index, test_index in kf.split(np.arange(num_patients)):
    print(train_index, test_index)

[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69] [0 1 2 3 4]
[ 0  1  2  3  4 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69] [5 6 7 8 9]
[ 0  1  2  3  4  5  6  7  8  9 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69] [10 11 12 13 14]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69] [15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52


In [34]:
cumulative_slices = np.cumsum([0] + slices_per_patient)

In [35]:
# cumulative_slices

In [36]:
fold = 0
losses = []
ious = []
iou_thresholds = []
histories = []

In [ ]:
for train_index, test_index in kf.split(np.arange(num_patients)):
    
    print(train_index, test_index)
    
    train_slices = []
    y_train_slices = []
    test_slices = []
    y_test_slices = []
    
    for i in train_index:
        start_idx = cumulative_slices[i]
        end_idx = cumulative_slices[i + 1]
        train_slices.extend(images_array[start_idx:end_idx])
        y_train_slices.extend(y_train_array[start_idx:end_idx])

    for i in test_index:
        start_idx = cumulative_slices[i]
        end_idx = cumulative_slices[i + 1]
        test_slices.extend(images_array[start_idx:end_idx])
        y_test_slices.extend(y_train_array[start_idx:end_idx])

    print(len(train_slices), len(y_train_slices))
    print(len(test_slices), len(y_test_slices))
    
    

    num_train_patients = len(train_index)
    train_patients_idx, val_patients_idx = train_test_split(
        np.arange(num_train_patients), test_size=0.1 #, shuffle=False
    )
    
    print(train_patients_idx,val_patients_idx, test_index)


    # get validation set from training set
    val_slices = []
    y_val_slices = []

    for i in val_patients_idx:
        start_idx = cumulative_slices[train_index[i]]
        end_idx = cumulative_slices[train_index[i] + 1]
        val_slices.extend(images_array[start_idx:end_idx])
        y_val_slices.extend(y_train_array[start_idx:end_idx])

        
        
    # update training set w/o validation set
    filtered_train_slices = []
    filtered_y_train_slices = []

    for i in train_patients_idx:
        start_idx = cumulative_slices[train_index[i]]
        end_idx = cumulative_slices[train_index[i] + 1]
        filtered_train_slices.extend(images_array[start_idx:end_idx])
        filtered_y_train_slices.extend(y_train_array[start_idx:end_idx])



    X_train_array = np.array(filtered_train_slices)
    y_train_array = np.array(filtered_y_train_slices)
    X_val_array = np.array(val_slices)
    y_val_array = np.array(y_val_slices)
    X_test_array = np.array(test_slices)
    y_test_array = np.array(y_test_slices)
    
    print(X_train_array.shape, y_train_array.shape)
    print(X_val_array.shape, y_val_array.shape)
    print(X_test_array.shape, y_test_array.shape)
    
    K.clear_session()

    model = C.Helper.create_unet(X_train_array[0].shape)

    print(f"Training fold {fold}...")

    
    history = model.fit(X_train_array, y_train_array, 
                        epochs=100, 
                        batch_size=16, 
                        validation_data=(X_val_array, y_val_array))
    
    
    histories.append(history.history)

    
    scores = model.evaluate(X_test_array, y_test_array, verbose=0)

    
    losses.append(scores[0])
    ious.append(scores[1])
    iou_thresholds.append(scores[2])
    
    fold += 1

[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69] [0 1 2 3 4]
15591 15591
1175 1175
[42 49 34 13 32 60  2 15 57  6 58 52  9 53 23 38 47 59 27 39 50 51  4 44
 25 64 31 33  7  0 29 45 54 21  5 62 55 61 30 12 28 11  1 63 19 18 10 16
 22 48 46 40 36  8 37  3 26 20] [41 17 14 56 43 35 24] [0 1 2 3 4]
(14119, 512, 512, 1) (14119, 512, 512, 1)
(1472, 512, 512, 1) (1472, 512, 512, 1)
(1175, 512, 512, 1) (1175, 512, 512, 1)
-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


2024-10-01 01:05:31.955230: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-10-01 01:05:32.543384: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.39GiB deviceMemoryBandwidth: 1.41TiB/s
2024-10-01 01:05:32.551748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:4e:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.39GiB deviceMemoryBandwidth: 1.41TiB/s
2024-10-01 01:05:32.551774: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-10-01 01:05:32.554809: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.

Training fold 0...


2024-10-01 01:05:40.121877: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-10-01 01:05:40.123352: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245750000 Hz


Epoch 1/100


2024-10-01 01:05:41.761911: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2024-10-01 01:05:42.248403: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2024-10-01 01:05:42.858819: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2024-10-01 01:05:42.858846: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2024-10-01 01:05:42.858908: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-10-01 01:05:42.924419: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-10-01 01:05:43.539017: I tensorflow/stream_exec

883/883 [==============================] - 408s 449ms/step - loss: 0.0693 - iou: 6.3762e-05 - iou_thresholded: 0.3516 - val_loss: 0.0018 - val_iou: 1.5232e-04 - val_iou_thresholded: 0.8829
Epoch 2/100
883/883 [==============================] - 390s 442ms/step - loss: 9.1518e-04 - iou: 5.9066e-04 - iou_thresholded: 0.4306 - val_loss: 4.9396e-04 - val_iou: 5.2362e-04 - val_iou_thresholded: 0.8829
Epoch 3/100
883/883 [==============================] - 390s 442ms/step - loss: 3.1731e-04 - iou: 0.0018 - iou_thresholded: 0.4034 - val_loss: 1.9597e-04 - val_iou: 0.0015 - val_iou_thresholded: 0.8829
Epoch 4/100
883/883 [==============================] - 390s 442ms/step - loss: 1.7032e-04 - iou: 0.0047 - iou_thresholded: 0.3881 - val_loss: 1.0350e-04 - val_iou: 0.0033 - val_iou_thresholded: 0.8830
Epoch 5/100
883/883 [==============================] - 390s 442ms/step - loss: 1.1177e-04 - iou: 0.0077 - iou_thresholded: 0.4073 - val_loss: 5.7400e-05 - val_iou: 0.0094 - val_iou_thresholded: 0.8871

In [ ]:
mean_loss = np.mean(losses)
mean_iou = np.mean(ious)
mean_iou_threshold = np.mean(iou_thresholds)

In [ ]:
print(f"Mean Loss: {mean_loss}")
print(f"Mean IoU: {mean_iou}")
print(f"Mean IoU Threshold: {mean_iou_threshold}")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Number of epochs
epochs = range(1, 101)

# List of metrics to plot
metrics = ['iou', 'val_iou', 'iou_thresholded', 'val_iou_thresholded', 'loss', 'val_loss']
metric_titles = ['IoU', 'Validation IoU', 'IoU Thresholded', 'Validation IoU Thresholded', 'Loss', 'Validation Loss']

# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 15))
axes = axes.flatten()

In [ ]:
for idx, metric in enumerate(metrics):
    for fold in range(11):
        axes[idx].plot(epochs, histories[fold][metric], label=f'Fold {fold+1}')
    axes[idx].set_title(metric_titles[idx])
    axes[idx].set_xlabel('Epochs')
    axes[idx].set_ylabel(metric_titles[idx])
    axes[idx].legend(loc='upper right', fontsize='small')
    axes[idx].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Number of epochs
epochs = range(1, 101)

# Plotting
plt.figure(figsize=(10, 6))

# Plot val_iou for fold 1
plt.plot(epochs, histories[0]['iou'], label='Fold 1 IoU')

# Plot val_iou for fold 2
plt.plot(epochs, histories[1]['iou'], label='Fold 2 IoU')

# Adding title and labels
plt.title('IoU over Epochs')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
# Number of epochs
epochs = range(1, 101)

# Plotting
plt.figure(figsize=(10, 6))

# Plot val_iou for fold 1
plt.plot(epochs, histories[0]['val_iou'], label='Fold 1 Validation IoU')

# Plot val_iou for fold 2
plt.plot(epochs, histories[1]['val_iou'], label='Fold 2 Validation IoU')

# Adding title and labels
plt.title('Validation IoU over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Validation IoU')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
# Number of epochs
epochs = range(1, 101)

# Plotting
plt.figure(figsize=(10, 6))

# Plot val_iou for fold 1
plt.plot(epochs, histories[0]['iou_thresholded'], label='Fold 1 IoU Thresholded')

# Plot val_iou for fold 2
plt.plot(epochs, histories[1]['iou_thresholded'], label='Fold 2 IoU Thresholded')

# Adding title and labels
plt.title('IoU Thresholded over Epochs')
plt.xlabel('Epochs')
plt.ylabel('IoU Thresholded')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
# Number of epochs
epochs = range(1, 101)

# Plotting
plt.figure(figsize=(10, 6))

# Plot val_iou for fold 1
plt.plot(epochs, histories[0]['loss'], label='Fold 1 loss')

# Plot val_iou for fold 2
plt.plot(epochs, histories[1]['loss'], label='Fold 2 loss')

# Adding title and labels
plt.title('loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
# Number of epochs
epochs = range(1, 101)

# Plotting
plt.figure(figsize=(10, 6))

# Plot val_iou for fold 1
plt.plot(epochs, histories[0]['val_loss'], label='Fold 1 Validation loss')

# Plot val_iou for fold 2
plt.plot(epochs, histories[1]['val_loss'], label='Fold 2 Validation loss')

# Adding title and labels
plt.title('Validation loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
# Number of epochs
epochs = range(1, 101)

# Plotting
plt.figure(figsize=(10, 6))

# Plot val_iou for fold 1
plt.plot(epochs, histories[0]['val_iou_thresholded'], label='Fold 1 Validation IoU Thresholded')

# Plot val_iou for fold 2
plt.plot(epochs, histories[1]['val_iou_thresholded'], label='Fold 2 Validation IoU Thresholded')

# Adding title and labels
plt.title('Validation IoU Thresholded over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Validation IoU Thresholded')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()